# SQL (SQLAlchemy)

>[Structured Query Language (SQL)](https://en.wikipedia.org/wiki/SQL) 是一种用于编程的领域特定语言，旨在管理关系数据库管理系统 (RDBMS) 中存储的数据，或用于关系数据流管理系统 (RDSMS) 中的流处理。它在处理结构化数据方面特别有用，即包含实体和变量之间关系的数据。

>[SQLAlchemy](https://github.com/sqlalchemy/sqlalchemy) 是一个开源的 `SQL` 工具包和对象关系映射器 (ORM)，适用于 Python 编程语言，遵循 MIT 协议发布。

本笔记本将介绍 `SQLChatMessageHistory` 类，该类允许将聊天记录存储在 `SQLAlchemy` 支持的任何数据库中。

请注意，要将其用于 `SQLite` 以外的数据库，您需要安装相应的数据库驱动程序。

## 设置

集成位于 `langchain-community` 包中，因此我们需要安装它。我们还需要安装 `SQLAlchemy` 包。

```bash
pip install -U langchain-community SQLAlchemy langchain-openai
```

以下是翻译后的内容：

在 LangSmith 上设置（可选，但推荐）以获得一流的可观测性。

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## 使用方法

要使用存储，您只需要提供两样东西：

1. Session Id - 会话的唯一标识符，例如用户名、邮箱、聊天 ID 等。
2. Connection string - 指定数据库连接的字符串。它将传递给 SQLAlchemy 的 create_engine 函数。

In [1]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="test_session", connection_string="sqlite:///sqlite.db"
)

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

In [2]:
chat_message_history.messages

[HumanMessage(content='Hello'), AIMessage(content='Hi')]

## 链式调用

我们可以轻松地将此消息历史类与 [LCEL Runnables](/docs/how_to/message_history) 结合使用。

为此，我们需要使用 OpenAI，因此需要安装它。

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI()

In [5]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [8]:
# This is where we configure the session id
config = {"configurable": {"session_id": "<SESSION_ID>"}}

In [9]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

AIMessage(content='Hello Bob! How can I assist you today?')

In [10]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)

AIMessage(content='Your name is Bob! Is there anything specific you would like assistance with, Bob?')